In [2]:
import numpy as np
import scipy
from numpy import linalg
import cvxopt
from cvxopt import solvers, matrix

def linear_kernel(x1, x2):
    return np.dot(x1, x2)

def polynomial_kernel(x, y, p=3):
    return (1 + np.dot(x, y)) ** p

def gaussian_kernel(x, y, sigma=5.0):
    return np.exp(-linalg.norm(x-y)**2 / (2 * (sigma ** 2)))

class OVA_SVM:
    kernel = 'rbf'
    estimators_ = None
    coefficients_ = None
    score = None
    
    X = []
    X_test = [] 
    y = []
    C = None # C-SVM parameter
    
    
    def __init__(self, C, kernel):
        self.C
        self.kernel = kernel
    def fit(self, X, y):
        self.X = X
        self.y = y
    def predict(X):
        self.X_test = X
        return 
    

ModuleNotFoundError: No module named 'cvxopt'

In [3]:
y = np.ones(10)
a = 
np.outer(y,y)

array([[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.]])

In [4]:
from sklearn.svm import LinearSVC

In [5]:
model = LinearSVC()

In [7]:
model.

AttributeError: 'LinearSVC' object has no attribute 'coefficients_'